#### <font color=red>2021/2/9 本版本<br>(1)部分底层数据重清洗，做到brand和goods中的一致;<br>(2)数据增强以后测试少样本品牌的ner预测;<br><br>下次版本<br>(1)加入天猫/淘宝/京东等第三方统计好的我们没有的品牌；<br>(2)确认所有品牌是真实有效的，进行人工核检;<br>(3)如果上面版本测试的数据增强的效果可以的话，则购买第三方完整的品牌数据后做数据增强;<br><br>下下次版本<br>(1)加入品类标签,CNT-B,CNT-I;<br>(2)制作品牌+国家的映射表，直接自动化处理dupont的国家的5个master字段；

In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from sqlalchemy import create_engine
import time,gc,os,re,sys,re,jieba,os,math
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
def check_cols(df):
    cols=df.columns.tolist()
    leng=int(len(cols)/5)+1
    for i in range(leng):
        tmp=df.iloc[:5,i*6:(i+1)*6]
        print(tmp.head())
        print('-'*30)

def check_miss_ratio(df):
    missdf=pd.isnull(df).sum().reset_index(name='miss_cnt')
    missdf.columns=['col','miss_cnt']
    missdf['miss_ratio']=missdf['miss_cnt']/df.shape[0]
    print(missdf.head(20))
    return missdf

In [ ]:
df=pd.DataFrame()
for i in range(11,44,11):
    tmp=pd.read_csv('d_platform_goods_%d.csv'%i,encoding='utf-8-sig')
    df=pd.concat([df,tmp],axis=0)
df=df.drop_duplicates().reset_index(drop=True)
print(df.shape)
print(df.sample(3))

In [ ]:
# path='./data/'
# files=os.listdir(path)
# print(files)

# brand=pd.read_excel(path+'d_master_brand.xls')
# cols=['master_brand_key', 'master_brand_name', 'master_brand_cn', 'master_brand_en', 'brand_company_cn', 'brand_company_en']
# brand=brand[cols].drop_duplicates().reset_index(drop=True)
# print(brand.shape)
# print(brand['master_brand_key'].nunique())
# print(brand.head(3))

# import_brand=pd.read_excel(path+'d_master_brand_import.xls')
# cols=['master_brand_key', 'master_brand_cn', 'master_brand_en', 'brand_company_cn', 'brand_company_en', 'cust_categ_l2_id',
#       'cust_categ_l2_cn', 'match_keywords_1', 'match_keywords_2', 'match_keywords_3', 'anti_match_keywords_1',
#       'anti_match_keywords_2', 'anti_match_keywords_3']
# import_brand=import_brand[cols].drop_duplicates().reset_index(drop=True)
# print(import_brand.shape)
# print(import_brand.head(3))
# print(pd.isnull(import_brand).sum())

In [9]:
print(df.shape)
print(df['platform_goods_id'].nunique())

(4522820, 23)
4519986


In [10]:
df=df.sort_values(by=['platform_goods_id','platform_goods_key'],ascending=True).drop_duplicates(subset=['platform_goods_id'],keep='last').reset_index(drop=True)

In [11]:
df=df[df['platform_goods_id'].notnull()]
print(df.shape)
print(df['platform_goods_key'].nunique())
print(df['platform_goods_id'].nunique())

(4519986, 23)
4519986
4519986


In [12]:
cols=['platform_goods_key', 'platform_goods_id', 'platform_goods_name',
      'platform_brand_key', 'platform_brand_name', 'master_brand_key', 'master_brand_name_cn', 'master_brand_name_en',
      'master_brand_company_cn', 'master_brand_company_en', 'platform_shop_key', 'platform_shop_name', 'platform_name_en',
      'platform_name_cn', 'platform_categ_key']
df=df[cols].drop_duplicates().reset_index(drop=True)
print(df.shape)
print(df['platform_goods_key'].nunique())
print(df['platform_goods_id'].nunique())

(4519986, 15)
4519986
4519986


In [13]:
missdf=check_miss_ratio(df)

                        col  miss_cnt  miss_ratio
0        platform_goods_key         0    0.000000
1         platform_goods_id         0    0.000000
2       platform_goods_name        55    0.000012
3        platform_brand_key    587245    0.129922
4       platform_brand_name    586988    0.129865
5          master_brand_key         0    0.000000
6      master_brand_name_cn         0    0.000000
7      master_brand_name_en         0    0.000000
8   master_brand_company_cn         0    0.000000
9   master_brand_company_en         0    0.000000
10        platform_shop_key         0    0.000000
11       platform_shop_name     32139    0.007110
12         platform_name_en         0    0.000000
13         platform_name_cn         0    0.000000
14       platform_categ_key         0    0.000000


In [14]:
df.loc[df['master_brand_name_cn'].str.contains('其他',na=False),'master_brand_name_cn']=np.nan
df.loc[df['master_brand_name_en'].str.contains('OTHERS',na=False),'master_brand_name_en']=np.nan
df.loc[df['platform_brand_name'].str.contains('其他',na=False),'platform_brand_name']=np.nan
df.loc[df['master_brand_company_cn'].str.contains('其他',na=False),'master_brand_company_cn']=np.nan
df.loc[df['master_brand_company_en'].str.contains('OTHERS',na=False),'master_brand_company_en']=np.nan

In [15]:
missdf=check_miss_ratio(df[['platform_goods_id','platform_goods_name', 'platform_brand_name', 'master_brand_name_cn', 'master_brand_name_en','master_brand_company_cn','master_brand_company_en']])

                       col  miss_cnt  miss_ratio
0        platform_goods_id         0    0.000000
1      platform_goods_name        55    0.000012
2      platform_brand_name    709833    0.157043
3     master_brand_name_cn   1291461    0.285722
4     master_brand_name_en   1291505    0.285732
5  master_brand_company_cn   1291462    0.285723
6  master_brand_company_en   1291506    0.285732


In [16]:
for col in ['platform_goods_name', 'platform_brand_name', 'master_brand_name_cn', 'master_brand_name_en','master_brand_company_cn','master_brand_company_en']:
    df.loc[df[col].notnull(),col]=df[df[col].notnull()][col].apply(lambda x:str(x).upper())

In [17]:
df=df[df['platform_goods_name'].notnull()]
print(df.shape)
print(df['platform_goods_id'].nunique())
print(df['platform_goods_key'].nunique())

(4519931, 15)
4519931
4519931


In [18]:
missdf=check_miss_ratio(df)

                        col  miss_cnt  miss_ratio
0        platform_goods_key         0    0.000000
1         platform_goods_id         0    0.000000
2       platform_goods_name         0    0.000000
3        platform_brand_key    587202    0.129914
4       platform_brand_name    709790    0.157036
5          master_brand_key         0    0.000000
6      master_brand_name_cn   1291416    0.285716
7      master_brand_name_en   1291460    0.285726
8   master_brand_company_cn   1291417    0.285716
9   master_brand_company_en   1291461    0.285726
10        platform_shop_key         0    0.000000
11       platform_shop_name     32139    0.007111
12         platform_name_en         0    0.000000
13         platform_name_cn         0    0.000000
14       platform_categ_key         0    0.000000


In [19]:
tmp=df.groupby('platform_categ_key')['platform_goods_key'].count().reset_index(name='cnt')
tmp=tmp.sort_values(by=['cnt'],ascending=False).reset_index(drop=True)
print(tmp.head(10))

   platform_categ_key     cnt
0                 594  139754
1                 602  137683
2                 151  135988
3                 597  134868
4                 596  122903
5                2363  117805
6                 162  103135
7                 861   91771
8                  53   90166
9                1125   87145


In [20]:
tmp=df.groupby('platform_categ_key')['master_brand_name_cn'].nunique().reset_index(name='cnt')
tmp=tmp.sort_values(by=['cnt'],ascending=False).reset_index(drop=True)
print(tmp.head(10))

   platform_categ_key   cnt
0                2363  1125
1                 596   990
2                 152   941
3                  53   910
4                 163   868
5                1125   819
6                1123   790
7                 151   764
8                 165   757
9                2380   744


In [21]:
tmp=df.groupby('master_brand_name_cn')['platform_categ_key'].nunique().reset_index(name='cnt')
tmp=tmp.sort_values(by=['cnt'],ascending=False).reset_index(drop=True)
print(tmp['cnt'].quantile([0,0.25,0.5,0.75,1]))
print(tmp.head(10))

0.00      1.0
0.25      3.0
0.50     11.0
0.75     32.0
1.00    167.0
Name: cnt, dtype: float64
  master_brand_name_cn  cnt
0                 伊蒂之屋  167
1                   迪奥  165
2                  美宝莲  162
3                  资生堂  156
4                   魅可  154
5                  欧莱雅  154
6                  香奈儿  153
7                  三熹玉  153
8                 雅诗兰黛  152
9                 悦诗风吟  148


In [22]:
print(tmp[tmp['cnt']==1]['master_brand_name_cn'].nunique())

355


In [23]:
print(tmp[tmp['cnt']==1]['master_brand_name_cn'].unique().tolist()[:10])

['雨潼', '醫美保健', '霏丝佳', 'RFB', 'SEA GLASS', 'SINIL PHARM', '霍氏', 'SANOFLORE', 'SUEE', '零语']


In [24]:
cols=['platform_goods_id','platform_goods_name','platform_brand_name', 'master_brand_name_cn','master_brand_name_en','platform_categ_key',
     'master_brand_company_cn','master_brand_company_en']
dff=df[cols].drop_duplicates().reset_index(drop=True)
dff.columns=['id','desc','brand','brand_cn','brand_en','cate_key','company_cn','company_en']
print(dff.shape)

(4519931, 8)


In [25]:
def clean_str(s):
    s=str(s)
    s=s.replace(' ','')
    return s.upper()

for col in ['desc','brand','brand_cn','brand_en','company_cn','company_en']:
    dff.loc[dff[col].notnull(),col]=dff[dff[col].notnull()][col].apply(clean_str)

dff.loc[dff['brand_cn'].isnull(),'is_brand_empty']=1
dff['is_brand_empty']=dff['is_brand_empty'].fillna(0)
print(dff['is_brand_empty'].value_counts())

0.0    3228515
1.0    1291416
Name: is_brand_empty, dtype: int64


In [26]:
def split_brand(df):
    df['brand_1']=df['brand'].apply(lambda x:'/'.join(x.split('（')))
    df['brand_1']=df['brand_1'].apply(lambda x:x.split('）')[0])
    df['brand_1']=df['brand_1'].apply(lambda x:'/'.join(x.split('(')))
    df['brand_1']=df['brand_1'].apply(lambda x:x.split(')')[0])

    df['brand_2']=df['brand_1'].apply(lambda x:x.split('/')[0])
    df['brand_3']=df['brand_1'].apply(lambda x:x.split('/')[1] if len(x.split('/'))>=2 else x.split('/')[0])

    pattern=re.compile('[\u4e00-\u9fa5]')

    df['brand_cn_1']=''
    df['brand_en_1']=''

    df['brand_2_mark']=df['brand_2'].apply(lambda x:re.sub(pattern,'NA',x))
    df.loc[df['brand_2_mark'].str.contains('NA'),'brand_cn_1']=df[df['brand_2_mark'].str.contains('NA')]['brand_2']

    df.loc[df['brand_cn_1']=='','brand_en_1']=df[df['brand_cn_1']=='']['brand_2']

    df['brand_cn_2']=''
    df['brand_en_2']=''

    df['brand_3_mark']=df['brand_3'].apply(lambda x:re.sub(pattern,'NA',x))
    df.loc[df['brand_3_mark'].str.contains('NA'),'brand_cn_2']=df[df['brand_3_mark'].str.contains('NA')]['brand_3']

    df.loc[df['brand_cn_2']=='','brand_en_2']=df[df['brand_cn_2']=='']['brand_3']

    df['brand_cn_fill']=df['brand_cn_1']
    df.loc[df['brand_cn_fill']=='','brand_cn_fill']=df[df['brand_cn_fill']=='']['brand_cn_2']
    df['brand_en_fill']=df['brand_en_1']
    df.loc[df['brand_en_fill']=='','brand_en_fill']=df[df['brand_en_fill']=='']['brand_en_2']

    for col in ['brand_en_fill','brand_en','brand_cn_fill','brand_cn','brand']:
        df.loc[df[col]=='',col]=np.nan
        df.loc[df[col]=='nan',col]=np.nan
        df.loc[df[col]=='NAN',col]=np.nan
#     need_cols=['desc','brand','brand_cn','brand_en','brand_key','brand_cn_fill','brand_en_fill','mark']
#     df=df[need_cols].drop_duplicates().reset_index(drop=True)
    df=df.drop_duplicates().reset_index(drop=True)
    gc.collect()
    return df

# for col in ['brand','brand_cn','brand_en']:
#     df[col]=df[col].astype(str)
# df1=split_brand(df)
# df1.sample(3)

# df.loc[(df['brand_cn'].isnull())&(df['brand_cn_fill'].notnull()),'brand_cn']=df[(df['brand_cn'].isnull())&(df['brand_cn_fill'].notnull())]['brand_cn_fill']
# df.loc[(df['brand_en'].isnull())&(df['brand_en_fill'].notnull()),'brand_en']=df[(df['brand_en'].isnull())&(df['brand_en_fill'].notnull())]['brand_en_fill']
# df.loc[(df['brand_cn_fill'].isnull())&(df['brand_cn'].notnull()),'brand_cn_fill']=df[(df['brand_cn_fill'].isnull())&(df['brand_cn'].notnull())]['brand_cn']
# df.loc[(df['brand_en_fill'].isnull())&(df['brand_en'].notnull()),'brand_en_fill']=df[(df['brand_en_fill'].isnull())&(df['brand_en'].notnull())]['brand_en']
# df.loc[(df['brand_cn'].isnull())&(df['brand_en'].notnull()),'brand_cn']=df[(df['brand_cn'].isnull())&(df['brand_en'].notnull())]['brand_en']
# df.loc[(df['brand_cn'].notnull())&(df['brand_cn_fill'].isnull()),'brand_cn_fill']=df[(df['brand_cn'].notnull())&(df['brand_cn_fill'].isnull())]['brand_cn']


In [27]:
def split_brand_1(v):
    pattern_en='[A-Z\'\-\:\°\.\’]'
    unit=re.findall(pattern_en,v)
    if len(unit)>0:
        en=''.join(unit)
    else:
        en=np.nan

    pattern_cn='[\u4e00-\u9fa5]'
    unit1=re.findall(pattern_cn,v)
    if len(unit1)>0:
        cn=''.join(unit1)
    else:
        cn=np.nan
    return en,cn

In [28]:
# s='DIOR迪奥'
# # s='迪奥（DIOR）'
# # s="D'IOR/迪奥 "
# a,b=split_brand_1(s)
# print(a,b)

In [29]:
dff['brand']=dff['brand'].astype(str)

In [30]:
dff['brand_en_fill']=dff['brand'].apply(lambda x:split_brand_1(x)[0])
dff['brand_cn_fill']=dff['brand'].apply(lambda x:split_brand_1(x)[1])

In [31]:
dff.loc[dff['brand']=='nan','brand']=np.nan

In [33]:
dff.loc[dff['brand']=='','brand']=np.nan

In [34]:
pd.isnull(dff).sum()

id                      0
desc                    0
brand              709790
brand_cn          1291416
brand_en          1291460
cate_key                0
company_cn        1291417
company_en        1291461
is_brand_empty          0
brand_en_fill     1605663
brand_cn_fill     1710093
dtype: int64

In [53]:
#处理比较特别的
dff1=dff[(dff['brand'].notnull())&(dff['brand_en_fill'].isnull())&(dff['brand_cn_fill'].isnull())]
print(dff1.shape)

dff2=dff[~dff['id'].isin(set(dff1['id']))]
print(dff2.shape)
print(len(dff1)+len(dff2)==len(dff))

(10993, 11)
(4508938, 11)
True


In [40]:
def split_brand_sep(v):
    pattern_dig='[0-9]'
    unit=re.findall(pattern_dig,v)
    if len(unit)>0:
        dig=''.join(unit)
    else:
        dig=np.nan
    return dig

In [46]:
s='1905化妆刷收纳袋套刷单支刷收纳袋EVA袋环保防尘防水'
res=split_brand_sep(s)
print(type(res))

<class 'str'>


In [47]:
s='化妆刷收纳袋套刷单支刷收纳袋EVA袋环保防尘防水'
res=split_brand_sep(s)
print(type(res))

<class 'float'>


In [55]:
dff1['brand_dig']=dff1['brand'].apply(lambda x:split_brand_sep(x))

In [56]:
# #余下的都是日文
# check=dff1[dff1['brand_dig'].isnull()]
# print(check['brand'].unique().tolist())

In [57]:
dt=pd.concat([dff1,dff2],axis=0)
print(len(dt)==len(dff))

True


In [59]:
dt.to_csv('data_1.csv',index=False,encoding='utf-8-sig')
gc.collect()

8

### <font color=red>others check

In [ ]:
# """检查补充缺失值的那些brand是否有用"""
# check=dff1[dff1['is_same']==1][['id','brand_cn_fill']].rename(columns={'brand_cn_fill':'brand_cn'})
# check['brand_en']=check['brand_cn']
# print(check['brand_cn'].nunique())
# print(check.shape)
# check.head(3)

# tmp1=dff2.groupby('brand_cn')['id'].nunique().reset_index(name='cn_cnt')
# tmp2=dff2.groupby('brand_en')['id'].nunique().reset_index(name='en_cnt')
# checks=check.merge(tmp1,on=['brand_cn'],how='left').merge(tmp2,on=['brand_en'],how='left')
# print(checks.head(3))

# pd.isnull(checks).sum()

# checks.loc[(checks['cn_cnt'].isnull())&(checks['en_cnt'].notnull()),'cn_cnt']=checks[(checks['cn_cnt'].isnull())&(checks['en_cnt'].notnull())]['en_cnt']

# pd.isnull(checks).sum()

# part1=checks[checks['cn_cnt']>=1]
# part2=checks[checks['cn_cnt'].isnull()]
# print(len(part1)+len(part2)==len(checks))

# print(len(part1)/len(checks))
# print(len(part1)/len(checks)*len(checks))
# print(len(part2)/len(checks))
# print(len(part2)/len(checks)*len(checks))

# tt=dff2['is_same'].value_counts().reset_index(name='cnt')
# tt.rename(columns={'index':'is_same'},inplace=True)
# tt['ratio']=tt['cnt']/len(df1)
# tt.head()

# """
# 1.主要查看了master_brand_cn和brand_cn_fill出现不一致的情况，主要原因是：
# (1)统一名称不一致，比如佳丽宝和嘉娜宝，相对来说，brand_cn_fill更加精确
# (2)处理下CHIOTURE稚优泉这样的情况
# (3)brand和series的区别，比如肌肤之钥和资生堂
# 结果：相对来说brand_cn_fill更加精确
# """

# check=dff2[dff2['is_same']==0]
# check[['desc','brand','brand_cn','brand_en','brand_cn_fill']].head(20)

# df2=df.copy()
# print(df2.shape)
# print(pd.isnull(df2).sum())

# """其中21.2%通过platform_brand_name进行补充缺失值的品牌的正确与否，其实取决于这个platform_brand_name这个字段的质量；
# 如何验证这个补缺以后的brand到底是不是正确呢？
# (1)划分为train+test,查看train中的正确率
# """

# train=df2[df2['mark']==0]
# valid=df2[(df2['mark']==1)&(df2['brand_cn_fill'].notnull())]
# test=df2[(df2['mark']==1)&(df2['brand_cn_fill'].isnull())]
# print(train.shape)
# print(valid.shape)
# print(test.shape)
# print(len(train)+len(valid)+len(test)==len(df2))

# def getLabel(sentence, loc_name, org_name):
#     label = []
#     for i in sentence:
#         if i in loc_name:
#             label.append('S')
#         elif i in org_name:
#             label.append('E')
#         else:
#             label.append('O')
#     return "".join(label)


# def re_ner(sentence, loc_name, org_name):
#     ne_list = []
#     label = getLabel(sentence, loc_name, org_name)
    
#     pattern = re.compile('SO*E')
#     ne_label = re.finditer(pattern, label)

#     for ne in ne_label:
#         ne_list.append(sentence[int(ne.start()):int(ne.end())])
#     return ','.join(ne_list)

# cn_brands=df2[df2['mark']==0]['brand_cn'].unique().tolist()
# print(len(cn_brands))

# test1=test.drop_duplicates()[:1000]

# test1['brand_cn_test']=''

# for i in tqdm_notebook(cn_brands):
#     s=i[0]
#     e=i[-1]
#     test1.loc[test1['brand_cn_test']=='','brand_cn_test']=test1[test1['brand_cn_test']=='']['desc'].apply(lambda x:re_ner(x,[s],[e]))
#     gc.collect()

# en_brands=df2[df2['mark']==0]['brand_en'].unique().tolist()
# print(len(en_brands))

# test1['brand_en_test']=''

# for i in tqdm_notebook(en_brands):
#     s=i[0]
#     e=i[-1]
#     test1.loc[test1['brand_en_test']=='','brand_en_test']=test1[test1['brand_en_test']=='']['desc'].apply(lambda x:re_ner(x,[s],[e]))
#     gc.collect()

# leng=1000
# c1=test1[test1['brand_cn_test']=='']
# print(len(c1)/leng)

# c2=test1[test1['brand_en_test']=='']
# print(len(c2)/leng)

# c3=test1[test1['brand_en_test']!='']
# c3[['desc','brand_cn_test','brand_en_test']].sample(5)

# test1['brand_cn_test_1']=''

# for i in tqdm_notebook(cn_brands):
#     test1.loc[(test1['brand_cn_test_1']=='')&(test1['desc'].str.contains(i)),'brand_cn_test_1']=i
#     gc.collect()

# test1['brand_en_test_1']=''
# for i in tqdm_notebook(en_brands):
#     test1.loc[(test1['brand_en_test_1']=='')&(test1['desc'].str.contains(i)),'brand_en_test_1']=i
#     gc.collect()

# leng=1000
# c1=test1[test1['brand_cn_test_1']=='']
# print(len(c1)/leng)

# c2=test1[test1['brand_en_test_1']=='']
# print(len(c2)/leng)

# """英文提取非常不正确，所以通过dict将英文全部转换为中文，然后提取中文brand"""

# cc=test1[test1['brand_cn_test_1']!='']
# print(cc[['desc','brand_cn_test_1']].head(20))

# s='L\OCC'
# if "\\" in s:
#     print('yes')

# all_cn_brands=df2[df2['mark']==0]['brand_cn'].unique().tolist()+df2[df2['mark']==0]['brand_cn_fill'].unique().tolist()
# all_cn_brands=list(set(all_cn_brands))
# for i in all_cn_brands:
#     if "\\" in i:
#         all_cn_brands.remove(i)
# print(len(all_cn_brands))

# all_en_brands=df2[df2['mark']==0]['brand_en'].unique().tolist()+df2[df2['mark']==0]['brand_en_fill'].unique().tolist()
# all_en_brands=list(set(all_en_brands))
# for i in all_en_brands:
#     if "\\" in i:
#         all_en_brands.remove(i)
# print(len(all_en_brands))

# test1['brand_cn_test_2']=''
# for i in tqdm_notebook(all_cn_brands):
#     test1.loc[(test1['brand_cn_test_2']=='')&(test1['desc'].str.contains(i)),'brand_cn_test_2']=i
#     gc.collect()

# leng=1000
# c11=test1[test1['brand_cn_test_2']=='']
# print(len(c11)/leng)

# check=test1[test1['brand_cn_test_2']!='']
# print(check[['desc','brand_cn_test_2']].head(30))

# print(train.shape)
# print(valid.shape)
# print(test.shape)

# testdf=test.copy()

# gc.collect()

# train_brand=train[['brand_cn','brand_en']].drop_duplicates()
# en_cn=dict(zip(train_brand['brand_en'],train_brand['brand_cn']))
# print(len(en_cn))

# testdf['desc_1']=testdf['desc']

# testdf=testdf.reset_index(drop=True)

# leng=len(testdf)
# batch=10000
# group=int(leng/batch)+1

# fill_rate=[]

# res=pd.DataFrame()
# for i in range(group):
#     print('BATCH:%d'%i)
#     start=i*batch
#     end=(i+1)*batch
#     if end>leng:
#         end=leng
#     tmp=testdf[start:end]
#     for en in tqdm_notebook(train['brand_en'].unique().tolist()):
#         tmp.loc[tmp['desc_1'].str.contains(en),'desc_1']=tmp[tmp['desc_1'].str.contains(en)]['desc_1'].apply(lambda x:x.replace(en,en_cn[en]))
#     for cn in tqdm_notebook(train['brand_cn'].unique().tolist()):
#         tmp.loc[tmp['desc_1'].str.contains(cn),'brand_cn_test_3']=cn
#         gc.collect()
#     rate=len(tmp[tmp['brand_cn_test_3'].notnull()])/len(tmp)
#     print(rate)
#     fill_rate.append(rate)
#     res=pd.concat([res,tmp[['id','desc','desc_1','brand_cn_test_3']]],axis=0)
#     print('-'*40)

# check1=res[res['brand_cn_test_3'].isnull()]
# check2=res[res['brand_cn_test_3'].notnull()]
# print(len(check1)+len(check2)==len(testdf))

# print(len(check2)/len(testdf))

# print(res[['desc_1','brand_cn_test_3']].head(20))